# Análisis: Función Objetivo vs Tamaño de Instancia

Este notebook analiza cómo se comporta el valor de la función objetivo (disposición total) a medida que aumenta el tamaño de las instancias.

**Función Objetivo:** Maximizar Σᵢ Σⱼ Σₜ xᵢⱼₜ · cᵢⱼₜ

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Carga de Datos

In [ ]:
# Cargar resumen de ejecución
with open('../resultados/resumen_ejecucion.json', 'r') as f:
    resumen = json.load(f)

# Extraer datos de instancias
datos = []

for tamano in ['small', 'medium', 'large']:
    carpeta = Path(f'../resultados/{tamano}')
    
    for archivo in carpeta.glob('resultado_*.json'):
        with open(archivo, 'r') as f:
            resultado = json.load(f)
            
            # Solo incluir instancias factibles
            if resultado['factible']:
                datos.append({
                    'id': resultado['id_instancia'],
                    'tipo': resultado['tipo'],
                    'trabajadores': resultado['trabajadores'],
                    'dias': resultado['dias'],
                    'valor_objetivo': resultado['valor_objetivo'],
                    'tamano_problema': resultado['trabajadores'] * resultado['dias']
                })

# Crear DataFrame
df = pd.DataFrame(datos)
df = df.sort_values('tamano_problema')

print(f"Total de instancias factibles: {len(df)}")
print(f"\nPrimeras filas:")
df.head()

In [ ]:
# Estadísticas por tamaño
print("Estadísticas por tipo de instancia:\n")
print(df.groupby('tipo')[['trabajadores', 'dias', 'tamano_problema', 'valor_objetivo']].describe().round(2))

## 2. Valor Objetivo vs Tamaño del Problema

In [ ]:
# Gráfico principal: Valor objetivo vs tamaño del problema
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

# Colores por tipo
colores = {'small': '#3498db', 'medium': '#e74c3c', 'large': '#2ecc71'}

for tipo in ['small', 'medium', 'large']:
    datos_tipo = df[df['tipo'] == tipo]
    ax.scatter(datos_tipo['tamano_problema'], 
               datos_tipo['valor_objetivo'],
               c=colores[tipo],
               s=150,
               alpha=0.7,
               label=tipo.capitalize(),
               edgecolors='black',
               linewidth=1.5)

# Línea de tendencia general
z = np.polyfit(df['tamano_problema'], df['valor_objetivo'], 1)
p = np.poly1d(z)
ax.plot(df['tamano_problema'], p(df['tamano_problema']), 
        "k--", alpha=0.5, linewidth=2, label='Tendencia lineal')

# Etiquetas
ax.set_xlabel('Tamaño del Problema (Trabajadores × Días)', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor de la Función Objetivo', fontsize=12, fontweight='bold')
ax.set_title('Función Objetivo vs Tamaño de la Instancia', fontsize=14, fontweight='bold', pad=20)
ax.legend(fontsize=11, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('graficos/objetivo_vs_tamano.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Pendiente de la tendencia: {z[0]:.2f}")
print(f"Intercepto: {z[1]:.2f}")

## 3. Análisis por Trabajadores y Días

In [ ]:
# Subgráficos: Valor objetivo vs trabajadores y vs días
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Valor objetivo vs número de trabajadores
for tipo in ['small', 'medium', 'large']:
    datos_tipo = df[df['tipo'] == tipo]
    axes[0].scatter(datos_tipo['trabajadores'], 
                    datos_tipo['valor_objetivo'],
                    c=colores[tipo],
                    s=120,
                    alpha=0.7,
                    label=tipo.capitalize(),
                    edgecolors='black',
                    linewidth=1.5)

axes[0].set_xlabel('Número de Trabajadores', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Valor Objetivo', fontsize=11, fontweight='bold')
axes[0].set_title('Valor Objetivo vs Trabajadores', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Valor objetivo vs número de días
for tipo in ['small', 'medium', 'large']:
    datos_tipo = df[df['tipo'] == tipo]
    axes[1].scatter(datos_tipo['dias'], 
                    datos_tipo['valor_objetivo'],
                    c=colores[tipo],
                    s=120,
                    alpha=0.7,
                    label=tipo.capitalize(),
                    edgecolors='black',
                    linewidth=1.5)

axes[1].set_xlabel('Número de Días', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Valor Objetivo', fontsize=11, fontweight='bold')
axes[1].set_title('Valor Objetivo vs Días', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('graficos/objetivo_trabajadores_dias.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Valor Objetivo Promedio por Tipo

In [ ]:
# Gráfico de barras con promedio por tipo
fig, ax = plt.subplots(figsize=(10, 6))

promedios = df.groupby('tipo')['valor_objetivo'].agg(['mean', 'std', 'min', 'max'])
promedios = promedios.reindex(['small', 'medium', 'large'])

x = range(len(promedios))
tipos = ['Small', 'Medium', 'Large']
colores_bar = ['#3498db', '#e74c3c', '#2ecc71']

bars = ax.bar(x, promedios['mean'], 
              color=colores_bar, 
              alpha=0.7, 
              edgecolor='black', 
              linewidth=1.5,
              yerr=promedios['std'],
              capsize=8,
              error_kw={'linewidth': 2, 'ecolor': 'gray'})

# Etiquetas en las barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_xlabel('Tipo de Instancia', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor Objetivo Promedio', fontsize=12, fontweight='bold')
ax.set_title('Valor Objetivo Promedio por Tipo de Instancia', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(tipos, fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('graficos/objetivo_promedio_tipo.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nValor objetivo promedio por tipo:")
print(promedios.round(2))

## 5. Análisis de Correlación

In [ ]:
# Matriz de correlación
import numpy as np

correlaciones = df[['trabajadores', 'dias', 'tamano_problema', 'valor_objetivo']].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlaciones, 
            annot=True, 
            fmt='.3f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={'shrink': 0.8},
            ax=ax)

ax.set_title('Matriz de Correlación', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('graficos/correlacion_matriz.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCorrelaciones con el valor objetivo:")
print(correlaciones['valor_objetivo'].sort_values(ascending=False))

## 6. Conclusiones

### Hallazgos principales:

1. **Relación lineal fuerte**: Existe una correlación positiva muy fuerte entre el tamaño del problema (trabajadores × días) y el valor de la función objetivo.

2. **Escalabilidad**: A medida que aumenta el número de trabajadores y días, el valor objetivo crece proporcionalmente, lo que indica que el modelo aprovecha eficientemente los recursos adicionales.

3. **Mayor impacto de trabajadores**: El número de trabajadores tiene mayor correlación con el valor objetivo que el número de días, sugiriendo que más trabajadores permiten asignaciones de mayor disposición.

4. **Variabilidad por tamaño**: Las instancias large muestran mayor variabilidad en el valor objetivo, lo cual es esperado dado que tienen más grados de libertad en las asignaciones.

5. **Instancias infactibles**: Las 2 instancias infactibles (12 y 15) son de tipo large, sugiriendo que restricciones más estrictas pueden hacer inviable la asignación en problemas grandes con demandas específicas.